# Continuous Bag of Words (CBOW) Model

In this exam, we will create a CBOW model using a sample Arabic traffic corpus. The corpus consists of sentences describing various traffic scenarios. The goal of the CBOW model is to predict a target word based on its surrounding context words.

# Importing Required Libraries

In this step, we import the necessary libraries to build and train the Continuous Bag of Words (CBOW) model.

- **TensorFlow and Keras**: Used to build the neural network model, including the layers like `Embedding`, `Dense`, and `Lambda`.
- **Tokenizer**: A utility from Keras for tokenizing and processing text data.
- **NumPy**: Used for handling numerical operations, particularly for processing arrays and data manipulation.

These libraries will provide the essential tools for text preprocessing and model development in the upcoming steps.

Add more if needed!


In [2]:
import re
import nltk
import nltk
nltk.download('punkt')
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Lambda
from tensorflow.keras.utils import to_categorical
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Preparing the Corpus

In this step, we initialize the corpus that will be used for training the Continuous Bag of Words (CBOW) model. The corpus consists of Arabic sentences, each of which describes different traffic scenarios.

- **Corpus**: A collection of traffic-related sentences in Arabic.

This step sets up the text data that we will use in the upcoming stages of tokenization and model training.

In [3]:
corpus = [
    "الطريق مزدحم اليوم بسبب الحادث المروري الذي حدث صباحاً ويؤدي إلى تأخير كبير في الوصول",
    "كان الطريق مزدحما للغاية والسيارات متوقفة تقريباً نتيجة الازدحام الشديد والحركة بطيئة جداً ولا تتحسن",
    "أنا أحب الذهاب إلى السوق في الصباح الباكر لتفادي الازدحام وشراء الخضروات الطازجة دون الانتظار في الطريق",
    "السيارات بطيئة بسبب الازدحام المروري في الشارع الرئيسي والتأخير في حركة المرور خلال ساعات الذروة",
    "هناك ازدحام مروري في الشارع بسبب أعمال البناء والحفريات التي تعطل حركة السيارات وتتسبب في تأخير كبير",
    "ازدحام السيارات يزداد في المساء عندما يبدأ الجميع بالعودة إلى منازلهم من العمل وتتوقف حركة المرور بالكامل",
    "الطريق السريع يشهد ازدحاما مستمرا خلال فترة الظهيرة بسبب الشاحنات الكبيرة التي تبطئ حركة السير",
    "الحافلات والسيارات عالقة في الازدحام المروري في المنطقة التجارية مما يؤدي إلى تأخير وصول الناس إلى وجهاتهم",
    "حركة المرور مزدحمة اليوم بسبب الفعاليات التي تقام في وسط المدينة مما يزيد من صعوبة الوصول إلى هناك",
    "كان من الصعب جدا القيادة على الطريق الرئيسي اليوم بسبب الازدحام الخانق الذي استمر طوال اليوم",
    "الطريق إلى المطار مزدحم اليوم بسبب الحوادث المتكررة والتأخيرات الكبيرة في حركة المرور على الطريق السريع",
    "الشارع مزدحم بالسيارات والحافلات الكبيرة مما يجعل التنقل بطيئًا جدًا ويزيد من وقت الوصول إلى العمل",
    "ازدحام السيارات في المدينة أصبح مشكلة كبيرة خاصة خلال ساعات الذروة حيث يصعب التحرك بسرعة",
    "تفاقم الازدحام في الطرق الجانبية بسبب إغلاق الطريق الرئيسي المؤدي إلى وسط المدينة لصيانة الجسر",
    "ازدحام مروري خانق يواجه السكان يوميًا خلال تنقلهم من وإلى العمل على الطرق السريعة المؤدية إلى المدينة",
    "التأخيرات المرورية اليوم ناجمة عن سوء الأحوال الجوية والضباب الذي يعيق الرؤية ويبطئ حركة السيارات",
    "حوادث السير المتكررة على الطريق الزراعي تؤدي إلى ازدحام مروري شديد وتأخير كبير في وصول السيارات",
    "كانت حركة السير اليوم غير منتظمة بسبب تنظيم حدث رياضي كبير أدى إلى إغلاق بعض الشوارع الرئيسية",
    "الأعمال الإنشائية في الشارع الرئيسي تسببت في اختناق مروري كامل وتباطؤ في حركة السيارات خلال النهار",
    "تراكم السيارات عند تقاطع الطرق الرئيسية أدى إلى ازدحام شديد وزيادة كبيرة في مدة الانتظار للوصول إلى الجهة المطلوبة"
]

In [6]:
# Preprocessing the corpus
def preprocess_text(text):
    # Normalize Arabic letters
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ؤ", "و", text)
    text = re.sub("ئ", "ي", text)

    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenize sentences
    tokens = nltk.word_tokenize(text)

    # Remove stopwords
    stopwords = set(nltk.corpus.stopwords.words('arabic'))
    tokens = [word for word in tokens if word not in stopwords]

    return tokens

# Apply preprocessing to each sentence in the corpus
corpus_preprocessed = [preprocess_text(sentence) for sentence in corpus]
print("\nPreprocessed corpus:")
print(corpus_preprocessed)


Preprocessed corpus:
[['الطريق', 'مزدحم', 'اليوم', 'بسبب', 'الحادث', 'المروري', 'حدث', 'صباحا', 'ويودي', 'الى', 'تاخير', 'كبير', 'الوصول'], ['الطريق', 'مزدحما', 'للغاية', 'والسيارات', 'متوقفة', 'تقريبا', 'نتيجة', 'الازدحام', 'الشديد', 'والحركة', 'بطيية', 'جدا', 'تتحسن'], ['انا', 'احب', 'الذهاب', 'الى', 'السوق', 'الصباح', 'الباكر', 'لتفادي', 'الازدحام', 'وشراء', 'الخضروات', 'الطازجة', 'الانتظار', 'الطريق'], ['السيارات', 'بطيية', 'بسبب', 'الازدحام', 'المروري', 'الشارع', 'الرييسي', 'والتاخير', 'حركة', 'المرور', 'خلال', 'ساعات', 'الذروة'], ['ازدحام', 'مروري', 'الشارع', 'بسبب', 'اعمال', 'البناء', 'والحفريات', 'تعطل', 'حركة', 'السيارات', 'وتتسبب', 'تاخير', 'كبير'], ['ازدحام', 'السيارات', 'يزداد', 'المساء', 'عندما', 'يبدا', 'الجميع', 'بالعودة', 'الى', 'منازلهم', 'العمل', 'وتتوقف', 'حركة', 'المرور', 'بالكامل'], ['الطريق', 'السريع', 'يشهد', 'ازدحاما', 'مستمرا', 'خلال', 'فترة', 'الظهيرة', 'بسبب', 'الشاحنات', 'الكبيرة', 'تبطي', 'حركة', 'السير'], ['الحافلات', 'والسيارات', 'عالقة', 'الازدحام', 'ال

In [7]:
# Tokenizing the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus_preprocessed)

# Convert words in the corpus into vectors of integers
sequences = tokenizer.texts_to_sequences(corpus_preprocessed)
print("\nSequences after converting words into vectors of integers:")
print(sequences)


Sequences after converting words into vectors of integers:
[[2, 16, 6, 3, 40, 17, 25, 41, 42, 1, 18, 10, 19], [2, 43, 44, 26, 45, 46, 47, 7, 48, 49, 27, 20, 50], [51, 52, 53, 1, 54, 55, 56, 57, 7, 58, 59, 60, 28, 2], [5, 27, 3, 7, 17, 11, 12, 61, 4, 13, 9, 29, 30], [8, 14, 11, 3, 62, 63, 64, 65, 4, 5, 66, 18, 10], [8, 5, 67, 68, 69, 70, 71, 72, 1, 73, 21, 74, 4, 13, 75], [2, 31, 76, 77, 78, 9, 79, 80, 3, 81, 22, 82, 4, 23], [83, 26, 84, 7, 17, 85, 86, 87, 1, 18, 32, 88, 1, 89], [4, 13, 90, 6, 3, 91, 92, 33, 15, 93, 94, 19, 1], [95, 20, 96, 2, 12, 6, 3, 7, 97, 98, 99, 6], [2, 1, 100, 16, 6, 3, 101, 34, 102, 22, 4, 13, 2, 31], [11, 16, 103, 104, 22, 105, 106, 107, 20, 108, 109, 19, 1, 21], [8, 5, 15, 110, 111, 35, 9, 29, 30, 112, 113, 114], [115, 7, 24, 116, 3, 36, 2, 12, 117, 1, 33, 15, 118, 119], [8, 14, 120, 121, 122, 123, 9, 124, 125, 21, 24, 126, 127, 1, 15], [128, 129, 6, 130, 131, 132, 133, 134, 135, 136, 137, 4, 5], [138, 23, 34, 2, 139, 140, 1, 8, 14, 37, 141, 10, 32, 5], [142,

# Defining Vocabulary and Model Parameters

In this step, we define key parameters that will be used to configure the CBOW model.

- **Vocabulary size**: We calculate the size of the vocabulary based on the number of unique words in the corpus. The `vocab_size` represents the total number of unique tokens (words) in the dataset plus one for padding.
  
- **Embedding size**: The `embedding_size` defines the dimensionality of the word embeddings. In this case, we set the embedding size to 10, meaning each word will be represented as a 10-dimensional vector in the embedding layer.

- **Window size**: The `window_size` defines how many words to the left and right of the target word are considered as context. Here, a window size of 2 means that two words before and two words after the target word will be used as context.

These parameters will play an essential role in shaping the CBOW model architecture.


In [9]:
# Vocabulary size: unique words + 1 for padding
vocab_size = len(tokenizer.word_index) + 1
print(f"Vocabulary size:  {vocab_size}")

# Embedding size and window size
embedding_size = 10
window_size = 2

Vocabulary size:  161


# Preparing Context-Target Pairs for CBOW

In this step, we generate the context-target pairs from the tokenized sequences to train the CBOW model.

- **Context words**: For each word in a sequence, the surrounding words (within the window size) are considered as context. The context consists of the words immediately before and after the target word.
  
- **Target word**: The word in the middle of the context window is treated as the target word that the model will learn to predict.

We iterate through each sequence, collecting the context words and corresponding target words:
- For each word in a sequence, we gather the surrounding words based on the defined window size.
- The middle word is the target, and the surrounding words form the context.

Finally:
- **`X`**: An array of context words.
- **`y`**: The target words are one-hot encoded, which means they are converted into a categorical format where each word is represented as a vector of length equal to the vocabulary size.

These context-target pairs will be used to train the CBOW model to predict a target word based on its context.


In [10]:
# Generate context and target pairs
contexts = []
targets = []

# Iterate over each sequence in sequences
for sequence in sequences:
    for i in range(window_size, len(sequence) - window_size):
        # Extract context: words before and after the target word
        context = sequence[i - window_size:i] + sequence[i + 1:i + window_size + 1]
        # Extract target word (the middle word in the window)
        target = sequence[i]
        contexts.append(context)
        targets.append(target)

# Convert context and targets into numpy arrays
X = np.array(contexts)
y = tf.keras.utils.to_categorical(targets, num_classes=vocab_size)

In [11]:
# Output shapes
print(f"Contexts shape: {X.shape}")
print(f"Targets shape (one-hot encoded): {y.shape}")

Contexts shape: (195, 4)
Targets shape (one-hot encoded): (195, 161)


In [12]:
X

array([[  2,  16,   3,  40],
       [ 16,   6,  40,  17],
       [  6,   3,  17,  25],
       [  3,  40,  25,  41],
       [ 40,  17,  41,  42],
       [ 17,  25,  42,   1],
       [ 25,  41,   1,  18],
       [ 41,  42,  18,  10],
       [ 42,   1,  10,  19],
       [  2,  43,  26,  45],
       [ 43,  44,  45,  46],
       [ 44,  26,  46,  47],
       [ 26,  45,  47,   7],
       [ 45,  46,   7,  48],
       [ 46,  47,  48,  49],
       [ 47,   7,  49,  27],
       [  7,  48,  27,  20],
       [ 48,  49,  20,  50],
       [ 51,  52,   1,  54],
       [ 52,  53,  54,  55],
       [ 53,   1,  55,  56],
       [  1,  54,  56,  57],
       [ 54,  55,  57,   7],
       [ 55,  56,   7,  58],
       [ 56,  57,  58,  59],
       [ 57,   7,  59,  60],
       [  7,  58,  60,  28],
       [ 58,  59,  28,   2],
       [  5,  27,   7,  17],
       [ 27,   3,  17,  11],
       [  3,   7,  11,  12],
       [  7,  17,  12,  61],
       [ 17,  11,  61,   4],
       [ 11,  12,   4,  13],
       [ 12,  

In [13]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

# Building and Training the CBOW Model

In this step, we build and train the Continuous Bag of Words (CBOW) model using the context-target pairs created earlier.

1. **Model architecture**:
   - **Embedding layer**: This layer transforms the input context words into dense vector representations (embeddings) of size defined by `embedding_size`. The `input_dim` is set to the vocabulary size, and the `input_length` is twice the window size (since context consists of words from both sides of the target).
   
   - **Lambda layer**: This layer computes the mean of the context word embeddings. It averages the embeddings of the context words to generate a single representation that will be used to predict the target word.
   
   - **Dense layer**: This fully connected layer outputs a probability distribution over the vocabulary, using the softmax activation function. It predicts the most likely target word based on the context word embeddings.

2. **Compilation**:
   The model is compiled using the Adam optimizer and categorical cross-entropy as the loss function, which is suitable for multi-class classification tasks. Accuracy is used as a metric to evaluate the model's performance during training.

3. **Training the model**:
   The model is trained on the context-target pairs for 500 epochs. During each epoch, the model learns to predict the target word based on the context, refining its weights to improve accuracy.

4. **Saving the model weights**:
   After training, the model weights are saved to a file (`cbow_model.weights.h5`) for future use. This allows us to load the trained model later without retraining.

By the end of this step, the CBOW model will have learned to predict target words based on their surrounding context from the given corpus.

In [21]:
# Define the CBOW model
model = Sequential()

# Embedding layer: Transforms context words into dense vector representations
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, input_length=window_size * 2))

# Lambda layer: Compute the mean of the context word embeddings
model.add(Lambda(lambda x: tf.reduce_mean(x, axis=1)))

# Dense layer: Outputs a probability distribution over the vocabulary (softmax)
model.add(Dense(vocab_size, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
# Train the model
model.fit(X, y, epochs=500, verbose=1)

Epoch 1/500
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0000e+00 - loss: 5.0813  
Epoch 2/500
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0423 - loss: 5.0765 
Epoch 3/500
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0314 - loss: 5.0739     
Epoch 4/500
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0663 - loss: 5.0697 
Epoch 5/500
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0918 - loss: 5.0660 
Epoch 6/500
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0959 - loss: 5.0619 
Epoch 7/500
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1228 - loss: 5.0575 
Epoch 8/500
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1174 - loss: 5.0552 
Epoch 9/500
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1286 - loss: 5.0507 
Epoch 10/500
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1102 - loss: 5.0479 
Epoch 11/500
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1396 - loss: 5.0428 
Epoch 12/500
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1

# Predicting a Word Using the Trained CBOW Model

In this step, we define a function to predict a word based on a given context using the trained CBOW model.

1. **Function: `predict_word`**:
   - **Input**: The function takes a list of context words as input. The number of context words should match the expected size (2 times the window size).
   - **Context sequence conversion**: The input context words are tokenized into a sequence of integers using the same tokenizer that was used during training.
   - **Input validation**: The function checks whether the length of the context sequence matches the expected size (2 times the window size). If not, it prints an error message.
   - **Prediction**: The tokenized context is fed into the trained CBOW model, which predicts the probability distribution over the vocabulary.
   - **Retrieve predicted word**: The predicted word is the one with the highest probability. The function retrieves the word corresponding to the predicted index from the tokenizer's word index.

2. **Example**:
   - We provide a sample context: `['الحادث', 'بسبب', 'مزدحم', 'الطريق']`.
   - The function predicts the word that fits best in this context, based on the model's learned weights.
   - The predicted word is printed along with the input context.

This function allows us to test the CBOW model by predicting words based on their surrounding context from the corpus.

In [25]:
def predict_word_simple(model, tokenizer, context_words, window_size):
    # Convert context words to integers
    context_sequence = [tokenizer.word_index.get(word, 0) for word in context_words]

    # Check if we have the right number of context words
    if len(context_sequence) != window_size * 2:
        print(f"Error: Please provide {window_size * 2} context words.")
        return None

    # Reshape to the format the model expects
    context_sequence = np.array([context_sequence])

    # Predict the target word
    prediction = model.predict(context_sequence)
    predicted_index = np.argmax(prediction)

    # Find the word corresponding to the predicted index
    for word, index in tokenizer.word_index.items():
        if index == predicted_index:
            return word

    return None  # Return None if no word is found

# Example usage
context = ['الحادث', 'بسبب', 'مزدحم', 'الطريق']  # Sample context
predicted_word = predict_word_simple(model, tokenizer, context, window_size)

if predicted_word:
    print(f"Context: {' '.join(context)}")
    print(f"Predicted word: {predicted_word}")
else:
    print("Prediction failed.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Context: الحادث بسبب مزدحم الطريق
Predicted word: اليوم
